In [56]:
import pickle
import torch
import tensorflow as tf
from kobert_transformers import get_kobert_model
import numpy as np
import pandas as pd
from kobert_transformers import get_tokenizer
import datetime

In [4]:
tf.debugging.set_log_device_placement(True) 
#코드 실행 전에 추가해주면 연산을 할때 어떤 장치에 할당 되었는지 확인할 수 있음

In [5]:
with open('body_token_id_padding.pkl', 'rb') as file:
    train_X = pickle.load(file)

In [6]:
train_x = np.array(train_X)

In [7]:
train_x.shape

(200000, 512)

In [8]:
pytorch_kobert_model = get_kobert_model()

In [9]:
def kobert_convert(integer_encoding, kobert_model):
  '''정수 인코딩된 자료를 pytorch kobert 모델 통과해서 tensorflow의 tensor로 변환'''
  return tf.convert_to_tensor(kobert_model(torch.LongTensor(integer_encoding))[0].detach())

In [10]:
with open('news_normalized.pickle', 'rb') as file:
    data = pickle.load(file)

In [11]:
data.head()

,id,title,date,body,summary,link,body_morphs,summary_morphs,site
0,1,"이총리 ""폭염, 특별재난 준해…전기요금 제한적 특별배려 검토""",2018.07.31. 오전 11:53,"국무회의 주재···체계적 폭염대책 주문""최저임금 명암···변화 수용하되 진통 최소화...",이낙연 국무총리는 31일 계속되는 폭염으로 전기요금 부담에 대한 우려가 커지고 있는...,https://news.naver.com/main/ranking/read.nhn?r...,"[국무회의, 주재···체계적, 폭염대책, 주문, 최저임금, 명암···변화, 수용, ...","[이낙연, 국무총리, 31일, 계속, 는, 폭염, 전기요금, 부담, 우려, 는, 관...",naver
1,2,"김성태 ""盧 탄핵때도 기무사 대응문건 의혹…즉시 제출하라""",2018.07.31. 오전 10:25,"""대통령, 여름휴가 때 노동자·소상공인 생각하길""김성태 자유한국당 원내대표. = 김...","김성태 자유한국당 원내대표는 31일 ""지난 2004년 노무현 전 대통령 탄핵 당시 ...",https://news.naver.com/main/ranking/read.nhn?r...,"[대통령, 여름휴가, 노동자·소상공, 생각하길""김성태, 자유한국당, 원내대표, =,...","[김성태, 자유한국당, 원내대표, 31, 2004년, 노무현, 대통령, 탄핵, 당시...",naver
2,3,"합참, 北 ICBM 제작 정황 보도에 ""면밀히 추적·감시중""",2018.07.31. 오전 11:29,"노재천 공보실장 ""공식확인 부적절···한미간 공조""워싱턴포스트 ""액화연료 사용 IC...",합동참모본부는 31일 북한이 대륙간탄도미사일을 만들고 있는 정황을 포착했다는 보도와...,https://news.naver.com/main/ranking/read.nhn?r...,"[노재천, 공보실장, 공식확인, 부적절···한미간, 공조""워싱턴포스트, 액화연료, ...","[합동참모본부, 31, 북한, 이, 대륙간탄도미사일, 는, 정황, 을, 포착, 었다...",naver
3,4,"기무사 ""盧 대통령 탄핵 당시 계엄검토 문건 작성 안해""",2018.07.31. 오후 2:43,김성태 의원 주장에 반박···지난 정부에서도 관련 내용 확인 국방부 특별수사단이 계...,국군기무사령부는 31일 노무현 전 대통령 탄핵 당시 계엄령 문건을 작성했다는 김성태...,https://news.naver.com/main/ranking/read.nhn?r...,"[김성태, 의원, 주장, 반박···지난, 정부, 에서도, 관련, 내용, 확인, 국방...","[국군기무사령부, 31일, 노무현, 대통령, 탄핵, 당시, 계엄령, 문건, 작성, ...",naver
4,5,"'종전문제 다룰까' 北, 이례적 회담 제의 이어 전향적 태도 보여",2018.07.31. 오후 12:55,"안익산 대표 "" 흔들어 종전선언? 보도, 그럴 수 있다"" 이해 안익산 북측 수석대표...",북한이 이례적으로 남북 장성급 군사회담을 먼저 제의한데 이어 회담에서 전향적인 태도...,https://news.naver.com/main/ranking/read.nhn?r...,"[안익산, 대표, 흔들, 종전선언, ?, 보도, 이해, 안익산, 북측, 수석대표, ...","[북한, 이례적, 남북, 장성급, 군사회담, 먼저, 제의한데, 이, 이, 회담, 전...",naver


## 메모리가없어...5만개로 줄여보기

In [12]:
data = data[:50000]

In [13]:
smorphs = data.summary_morphs

In [14]:
smorphs = smorphs.apply(lambda x : ['sos']+x+['eos'])

In [15]:
smorphs.head()

0    [sos, 이낙연, 국무총리, 31일, 계속, 는, 폭염, 전기요금, 부담, 우려,...
1    [sos, 김성태, 자유한국당, 원내대표, 31, 2004년, 노무현, 대통령, 탄...
2    [sos, 합동참모본부, 31, 북한, 이, 대륙간탄도미사일, 는, 정황, 을, 포...
3    [sos, 국군기무사령부, 31일, 노무현, 대통령, 탄핵, 당시, 계엄령, 문건,...
4    [sos, 북한, 이례적, 남북, 장성급, 군사회담, 먼저, 제의한데, 이, 이, ...
Name: summary_morphs, dtype: object

In [16]:
l=[]
m = 200
cnt= 0
for n,_ in enumerate(smorphs):
    if len(_)>m:
        l.append(n)
        cnt+=1

In [17]:
m #maximum length

200

In [18]:
cnt

186

In [19]:
l[:5]

[289, 384, 1086, 1194, 1355]

In [20]:
tok = get_tokenizer()

In [21]:
summarytok = []
for _ in smorphs:
    el_id = tok.convert_tokens_to_ids(_)
    summarytok.append(el_id[:200])       

In [22]:
hi = set()
for lst in summarytok:
    for _ in lst:
        if _ not in hi:
            hi.add(_)

In [23]:
max(hi) #max가 8001이다

8001

In [24]:
hi2 = set()
for lst in train_x:
    for _ in lst:
        if _ not in hi:
            hi2.add(_)

In [25]:
max(hi2) #본문내 max는 7999

7999

sos=2, eos=3

In [26]:
summarytok = []
for _ in smorphs:
    el_id = tok.convert_tokens_to_ids(_)
    el_id[0] = 2 #sos
    el_id[-1] = 3 #eos
    if(len(el_id)<200):
        new = el_id+[0]*(200-len(el_id))
    elif(len(el_id)>200):
        new = el_id[:200]
        new[-1] = 3 #eos
    summarytok.append(new)       

In [27]:
print(len(summarytok[1]))
print(summarytok[1])

200
[2, 0, 0, 0, 147, 0, 0, 5821, 0, 0, 0, 0, 0, 7782, 8, 5439, 0, 7782, 6888, 5585, 0, 5760, 0, 6556, 0, 5511, 6903, 0, 0, 0, 0, 6903, 0, 0, 7096, 0, 6210, 0, 0, 0, 7157, 7096, 0, 6157, 0, 5760, 0, 6544, 5886, 0, 0, 7088, 0, 0, 6888, 54, 5585, 0, 0, 5760, 8, 5725, 7207, 5821, 0, 0, 0, 5516, 0, 0, 0, 6903, 0, 7088, 0, 7782, 0, 0, 7096, 0, 5330, 5439, 0, 8, 0, 0, 6484, 0, 0, 5859, 5507, 0, 0, 6903, 0, 6855, 7782, 0, 8, 5439, 0, 7782, 6888, 54, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [28]:
print(len(summarytok[289]))
print(summarytok[289])

200
[2, 0, 0, 0, 0, 0, 0, 0, 0, 6634, 7096, 0, 0, 0, 0, 0, 0, 0, 0, 7095, 0, 0, 5477, 0, 0, 15, 0, 0, 0, 7088, 7945, 7782, 0, 6888, 0, 7086, 0, 0, 0, 7096, 5788, 0, 15, 6005, 0, 7095, 0, 0, 6896, 0, 7095, 0, 7096, 0, 0, 0, 5886, 0, 0, 7088, 0, 7096, 7295, 0, 6116, 0, 8, 0, 5886, 0, 0, 6896, 0, 0, 7289, 6896, 0, 0, 0, 7096, 0, 5886, 0, 6896, 0, 7788, 0, 6421, 7157, 0, 0, 6645, 7088, 0, 6855, 0, 7096, 0, 0, 6116, 7945, 7782, 0, 7088, 0, 7788, 8, 5439, 6300, 7782, 6888, 54, 0, 7086, 8, 0, 6366, 0, 0, 0, 7086, 6867, 6896, 5454, 5886, 0, 0, 7096, 0, 15, 0, 6271, 0, 0, 15, 7096, 0, 15, 7903, 0, 5477, 0, 15, 7096, 6889, 0, 0, 5760, 0, 0, 0, 6116, 0, 6896, 7012, 7782, 6855, 15, 6258, 0, 0, 15, 0, 6016, 0, 7078, 0, 6356, 0, 6116, 0, 7782, 6886, 46, 6165, 7096, 0, 5886, 0, 334, 6116, 0, 7782, 6855, 0, 7088, 0, 7782, 6892, 46, 0, 7012, 0, 0, 5886, 0, 0, 7096, 5561, 3]


In [29]:
hi = set()
for lst in summarytok:
    for _ in lst:
        if _ not in hi:
            hi.add(_)

In [30]:
len(hi)

2044

In [31]:
decoder_output = np.array(summarytok)

In [32]:
decoder_output.shape

(50000, 200)

In [33]:
MAX_LEGNTH = 512 #for padding
NUM_WORDS = 2064 #본문 기사 내 가장 많이 사용된 3000단어?(or 전체 단어수로 할지 미정)
VECTOR_SIZE = 768
MAX_SUMMARIZATION_LENGTH = 200

In [34]:
mapping = dict()
for n,_ in enumerate(hi):
    mapping[n] = _

In [35]:
print(mapping)

{0: 0, 1: 2, 2: 3, 3: 5, 4: 6, 5: 8, 6: 11, 7: 15, 8: 17, 9: 41, 10: 42, 11: 45, 12: 46, 13: 47, 14: 49, 15: 50, 16: 53, 17: 54, 18: 55, 19: 56, 20: 57, 21: 59, 22: 60, 23: 63, 24: 65, 25: 68, 26: 71, 27: 73, 28: 75, 29: 77, 30: 79, 31: 81, 32: 82, 33: 83, 34: 85, 35: 86, 36: 87, 37: 88, 38: 89, 39: 90, 40: 91, 41: 93, 42: 97, 43: 101, 44: 104, 45: 105, 46: 106, 47: 107, 48: 108, 49: 109, 50: 110, 51: 111, 52: 112, 53: 113, 54: 114, 55: 115, 56: 116, 57: 119, 58: 122, 59: 124, 60: 127, 61: 128, 62: 129, 63: 130, 64: 131, 65: 132, 66: 133, 67: 134, 68: 135, 69: 136, 70: 137, 71: 138, 72: 139, 73: 140, 74: 141, 75: 142, 76: 144, 77: 145, 78: 146, 79: 147, 80: 148, 81: 149, 82: 150, 83: 151, 84: 152, 85: 153, 86: 154, 87: 155, 88: 157, 89: 160, 90: 161, 91: 162, 92: 163, 93: 164, 94: 165, 95: 166, 96: 167, 97: 168, 98: 169, 99: 170, 100: 171, 101: 172, 102: 175, 103: 176, 104: 177, 105: 178, 106: 179, 107: 180, 108: 181, 109: 182, 110: 183, 111: 184, 112: 185, 113: 186, 114: 187, 115: 188

In [36]:
list(mapping.keys())[list(mapping.values()).index(3)]

2

In [3]:
decoder_target = np.zeros((50000,200,2044),dtype='float32')

In [37]:
for i, target in enumerate(summarytok):
    for n, word in enumerate(target):
        decoder_target[i,n,list(mapping.keys())[list(mapping.values()).index(word)]]=1

In [38]:
decoder_target[0]

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [39]:
MAX_LEGNTH = 512
NUM_WORDS = 2064 #본문 기사 내 가장 많이 사용된 3000단어?(or 전체 단어수로 할지 미정)
VECTOR_SIZE = 768
MAX_SUMMARIZATION_LENGTH = 200

In [40]:
decoder_input = np.array(summarytok)

In [41]:
decoder_input.shape

(50000, 200)

In [42]:
decoder_target.shape

(50000, 200, 2044)

In [43]:
train_x = train_x[:50000]

In [44]:
train_x.shape

(50000, 512)

In [58]:
from keras import Model
from keras.layers import Embedding, LSTM, Dropout, Input, Dense, dot, concatenate
from keras.optimizers import Adam

In [73]:
class Seq2Seq_Attention():
    def __init__(self):
        self.max_length = 512
        self.VECTOR_SIZE = 768
        self.max_summary_length = 200
        self.num_words = 2044
        
        self.sos =  2
        self.eos =  3
        
        self.optimizer = Adam()
        
        encoder_inputs = Input(shape=(self.max_length,self.VECTOR_SIZE))
        decoder_inputs = Input(shape=(self.max_summary_length,self.VECTOR_SIZE))
        
        self.att = self.build_att()

        self.att.compile(loss='categorical_crossentropy', optimizer=self.optimizer)
        print(self.att.summary())
        
        
    def build_att(self):
        def lstm(inputs, hs, seq=True, initial=None):
            output,h,c = LSTM(hs, return_state=True, return_sequences=seq)(inputs, initial_state=initial) 
            #return_seq=False & return_state=True: return only last h, c
            #return_seq=Ture & return_state=True: return all h, c
            return output, h, c
        
        def fc(n_h_layers, inputs, hn):
            for _ in range(n_h_layers):
                d = Dense(hn, activation='tanh')(inputs)
            if n_h_layers==0: 
                d = inputs
            output = Dense(hn, activation='softmax')(d)
            #모든 2064개 단어에 대한 확률값 (해당 위치에서의)
            return output
        
        #(encoder input) already embedded from koBERT(vector size 만큼)
        encoder_inputs = Input(shape=(self.max_length,self.VECTOR_SIZE))
        encoder_outputs, h, c = lstm(encoder_inputs, 256)  #Discard encoder outputs
        print(encoder_outputs)
        init_states = [h,c]
        
        decoder_inputs = Input(shape=(self.max_summary_length,self.VECTOR_SIZE))
        decoder_outputs, _, _ = lstm(decoder_inputs, 256, initial=init_states) #Discard encoder outputs
        
        value = Dense(5000, activation='tanh')(encoder_outputs)
        query = Dense(5000, activation='tanh')(decoder_outputs)
        print(value, query) #300x5000 두개
        
        attention = dot([query, value],axes=[2,2])
        print(attention) #300x300
        
        attention_softmaxed = fc(0, attention, self.max_length)
        print(attention_softmaxed) #300x300
        print(encoder_outputs)
        
        weighted = dot([attention_softmaxed, encoder_outputs], axes=[2,1]) #give weights to encoder outputs(=각 토큰)
        print(attention_softmaxed)
        
        print(weighted)
        
        decoder_for_final = concatenate([weighted, decoder_outputs]) #weighted token(2064x256) + decoder output(2064x256)
        #or add? 둘중에 성능좋은거 고르기
        
        decoder_final = fc(1, decoder_for_final, self.num_words)
        print(decoder_final)
        
        mod = Model([encoder_inputs, decoder_inputs], decoder_final) 
        return mod
    
    #Get data of batch size
    def load_batch(self, batch_size=32):
        self.n_batches = int(55000/32)
        for i in range(self.n_batches):
            batch_e = train_x[i*batch_size:(i+1)*batch_size]
            batch_d = decoder_input[i*batch_size:(i+1)*batch_size]
            batch_d_o = decoder_target[i*batch_size:(i+1)*batch_size]

            enc_ins, dec_ins, dec_out = [],[],[]
            for enc_in, dec_in, dec_o in zip(batch_e, batch_d, batch_d_o):
                enc_ins.append(enc_in)
                dec_ins.append(dec_in)
                dec_out.append(dec_o)

            yield enc_ins, dec_ins, dec_out
    
    #train model
    def train(self, epochs, batch_size = 32):
        start_time = datetime.datetime.now()
        
        for epoch in range(epochs):
            for batch_i, (enc_batch, dec_batch, dec_out_batch) in enumerate(self.load_batch(batch_size)):
                
                enc_batch = kobert_convert(enc_batch, pytorch_kobert_model)
                dec_batch = kobert_convert(dec_batch, pytorch_kobert_model)
                
                print(np.array(enc_batch).shape, np.array(dec_batch).shape, np.array(dec_out_batch).shape)
                
                enc_batch = np.array(enc_batch)
                dec_batch = np.array(dec_batch)
                dec_out_batch = np.array(dec_out_batch)
                
                att_loss = self.att.train_on_batch([enc_batch, dec_batch], dec_out_batch)
                
                elapsed_time = datetime.datetime.now() - start_time
                
                print("[Epoch %d/%d] [Batch %d/%d] Loss: %05f Time:%s"%(epoch, epochs, batch_i, self.n_batches, att_loss, elapsed_time))

In [74]:
model = Seq2Seq_Attention()

Tensor("lstm_10/Identity:0", shape=(None, 512, 256), dtype=float32)
Tensor("dense_25/Identity:0", shape=(None, 512, 5000), dtype=float32) Tensor("dense_26/Identity:0", shape=(None, 200, 5000), dtype=float32)
Tensor("dot_10/Identity:0", shape=(None, 200, 512), dtype=float32)
Tensor("dense_27/Identity:0", shape=(None, 200, 512), dtype=float32)
Tensor("lstm_10/Identity:0", shape=(None, 512, 256), dtype=float32)
Tensor("dense_27/Identity:0", shape=(None, 200, 512), dtype=float32)
Tensor("dot_11/Identity:0", shape=(None, 200, 256), dtype=float32)
Tensor("dense_29/Identity:0", shape=(None, 200, 2044), dtype=float32)
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 512, 768)]   0                                            
____________________________________________________________________

In [ ]:
model.train(epochs=10, batch_size=32)

(32, 512, 768) (32, 200, 768) (32, 200, 2044)
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/t